In [1]:
import numpy as np
import pandas as pd
import time
from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt


In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv("C:/Users/david/ml-100k/u.data",sep = '\t',names = header)

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

In [5]:
print n_users;n_items

943


1682L

In [6]:
 #将样本分为训练集与测试机
train_data,test_data = cv.train_test_split(df,test_size = 0.25)

In [24]:
train_data_matrix = np.zeros((n_users,n_items))
for line in train_data.itertuples():
        train_data_matrix[line[1]-1, line[2]-1] = line[3]

In [25]:
test_data_matrix = np.zeros((n_users,n_items))
for line in test_data.itertuples():
        test_data_matrix[line[1]-1,line[2]-1] = line[3]

In [26]:
#计算user相似矩阵与item相似矩阵,大小分别为943*943,1682*1682
user_similar = pairwise_distances(train_data_matrix, metric = "cosine")
item_similar = pairwise_distances(train_data_matrix.T, metric = "cosine")

In [28]:
user_similar[:2]

array([[ -2.22044605e-16,   8.85840141e-01,   9.70935046e-01, ...,
          8.73437052e-01,   8.81433950e-01,   6.83521783e-01],
       [  8.85840141e-01,  -4.44089210e-16,   8.81360820e-01, ...,
          9.23353898e-01,   8.62223498e-01,   9.29341543e-01]])

In [30]:

def read_file():

    header = ['user_id', 'item_id', 'rating', 'timestamp']
    df = pd.read_csv("C:/Users/david/ml-100k/u.data",sep = '\t',names = header)
    #去重之后得到一个元祖，分别表示行与列,大小分别为943与1682
    n_users = df.user_id.unique().shape[0]
    n_items = df.item_id.unique().shape[0]

    print 'all users is :' + str(n_users) + ', all items is :' + str(n_items)

    #将样本分为训练集与测试机
    train_data,test_data = cv.train_test_split(df,test_size = 0.25)

    train_data_matrix = np.zeros((n_users,n_items))
    for line in train_data.itertuples():
        train_data_matrix[line[1]-1, line[2]-1] = line[3]

    test_data_matrix = np.zeros((n_users,n_items))
    for line in test_data.itertuples():
        test_data_matrix[line[1]-1,line[2]-1] = line[3]

    #计算user相似矩阵与item相似矩阵,大小分别为943*943,1682*1682
    user_similar = pairwise_distances(train_data_matrix, metric = "cosine")
    item_similar = pairwise_distances(train_data_matrix.T, metric = "cosine")

    return (train_data_matrix,test_data_matrix,user_similar,item_similar)


In [31]:
train_data_matrix,test_data_matrix,user_similar,item_similar = read_file()
print 'user_similar.shape is :',user_similar.shape
print 'item_similar.shape is :',item_similar.shape

all users is :943, all items is :1682
user_similar.shape is : (943L, 943L)
item_similar.shape is : (1682L, 1682L)


In [32]:
def predict(rating, similar, type = 'user'):
    if type == 'user':
        mean_user_rating = rating.mean(axis = 1)
        rating_diff = (rating - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:,np.newaxis] + similar.dot(rating_diff) / np.array([np.abs(similar).sum(axis=1)]).T
    elif type == 'item':
        pred = rating.dot(similar) / np.array([np.abs(similar).sum(axis=1)])

    return pred


In [33]:
user_prediction = predict(train_data_matrix, user_similar, type = 'user')
item_prediction = predict(train_data_matrix, item_similar, type = 'item')

In [34]:


def rmse(prediction,ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))



In [35]:
print 'User based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))
print 'Item based CF RMSe: ' + str(rmse(item_prediction, test_data_matrix))

User based CF RMSE: 3.1228457173
Item based CF RMSe: 3.45058045945
